<a href="https://colab.research.google.com/github/beve0x/DRL-stock_trading/blob/main/FinRL_Ensemble_StockTrading_ICAIF_2020_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/2-Advance/FinRL_Ensemble_StockTrading_ICAIF_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading Using Ensemble Strategy

Tutorials to use OpenAI DRL to trade multiple stocks using ensemble strategy in one Jupyter Notebook | Presented at ICAIF 2020

* This notebook is the reimplementation of our paper: Deep Reinforcement Learning for Automated Stock Trading: An Ensemble Strategy, using FinRL.
* Check out medium blog for detailed explanations: https://medium.com/@ai4finance/deep-reinforcement-learning-for-automated-stock-trading-f1dad0126a02
* Please report any issues to our Github: https://github.com/AI4Finance-LLC/FinRL-Library/issues
* **Pytorch Version**



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)             

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [ ]:
# ## install finrl library
!pip install wrds
!pip install swig
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 27.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of th

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.4 MB/s eta 0:00:00
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...
Selecting previously unselected package libgl1-mesa-glx:amd64.
(Reading database ... 121749 files and directories currently installed.)
Preparing to unpack .../libgl1-mesa-glx_23.0.4-0ubuntu1~22.04.1_amd64.deb ...
Unpacking libgl1-mesa-glx:amd64 (23.0.4-0ubuntu1~22.04.1) ...
Selecting previously unselected package swig4.0.
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubuntu1_all.deb ...
Unpacking swig (4.0.2-1ubuntu1) ...
Setting up libgl1-mesa-glx:amd64 (23.0.4-0ubuntu1~22.04.1) ...
Setting up swig4.0 (4.0.2-1ubuntu1) ...
Setting up swig (4.0


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them.
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

<a id='1.4'></a>
## 2.4. Create Folders

In [ ]:
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [ ]:
print(DOW_30_TICKER)

NameError: name 'DOW_30_TICKER' is not defined

In [ ]:
pip install yahoo_fin --upgrade


In [ ]:
import pandas as pd
from yahoo_fin.stock_info import get_stats_valuation

import yahoo_fin.stock_info as si
dow_list = si.tickers_dow()

dow_stats = {}
for ticker in DOW_30_TICKER:
    temp = si.get_stats_valuation(ticker)
    temp = temp.iloc[:,:2]
    temp.columns = ["Attribute", "Recent"]
    dow_stats[ticker] = temp
dow_stats

combined_stats = pd.concat(dow_stats)
combined_stats = combined_stats.reset_index()
combined_stats

del combined_stats["level_1"]
# update column names
combined_stats.columns = ["Ticker", "Attribute", "Recent"]
combined_stats

pe_ratios = combined_stats[combined_stats["Attribute"]=="Enterprise Value/EBITDA"].reset_index()
pe_ratios

,index,Ticker,Attribute,Recent
0,8,AXP,Enterprise Value/EBITDA,NaN
1,17,AMGN,Enterprise Value/EBITDA,19.49
2,26,AAPL,Enterprise Value/EBITDA,21.67
3,35,BA,Enterprise Value/EBITDA,69.71
4,44,CAT,Enterprise Value/EBITDA,12.11
5,53,CSCO,Enterprise Value/EBITDA,10.08
6,62,CVX,Enterprise Value/EBITDA,9.76
7,71,GS,Enterprise Value/EBITDA,NaN
8,80,HD,Enterprise Value/EBITDA,15.72
9,89,HON,Enterprise Value/EBITDA,15.41


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Define the tickers and the date range
tickers = DOW_30_TICKER  # Example tickers, replace with DOW_30_TICKER or your list
start_date = datetime(2019, 12, 31)
end_date = datetime(2024, 1, 1)

# Initialize a DataFrame to store the EV/EBITDA ratios
ev_ebitda_ratios = pd.DataFrame()

current_date = start_date
while current_date <= end_date:
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        try:
            hist = stock.history(start=current_date, end=current_date + timedelta(days=1))
            if not hist.empty:
                # Placeholder for your logic to calculate EV and EBITDA
                # For example:
                # ev = calculate_ev_function(hist)
                # ebitda = calculate_ebitda_function(hist)
                # ev_ebitda_ratio = ev / ebitda if ebitda else None
                # Append the results to the DataFrame
                # ev_ebitda_ratios = ev_ebitda_ratios.append({'Date': current_date, 'Ticker': ticker, 'EV/EBITDA': ev_ebitda_ratio}, ignore_index=True)
                pass  # Remove this line once you add your calculation logic
        except Exception as e:
            print(f"Error retrieving data for {ticker} on {current_date}: {e}")
    current_date += timedelta(days=1)

# Display the results
print(ev_ebitda_ratios)

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


NameError: name 'DOW_30_TICKER' is not defined

Extracting Industry information from Dow 30 companies


In [ ]:
import yfinance as yf
import pandas as pd

# DOW_30_TICKER is your list of tickers
DOW_30_TICKER = ['AAPL', 'MSFT', 'IBM', 'WMT', 'V', 'PG', 'JNJ', 'UNH', 'INTC', 'XOM', 'GS', 'JPM', 'HD', 'MCD', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'MMM', 'MRK', 'AXP', 'TRV', 'UTX', 'VZ', 'WBA', 'PFE', 'NKE', 'DWDP']

# Prepare a list to store the ticker and industry information
ticker_industry_pairs = []

# Fetch industry for each ticker
for ticker in DOW_30_TICKER:
    t = yf.Ticker(ticker)
    info = t.info
    industry = info.get('industry', 'N/A')  # Use 'N/A' if industry information is not available
    ticker_industry_pairs.append([ticker, industry])

# Convert the list to a DataFrame
industry_df = pd.DataFrame(ticker_industry_pairs, columns=['Ticker', 'Industry'])

# Display the DataFrame
print(industry_df)

   Ticker                             Industry
0    AAPL                 Consumer Electronics
1    MSFT            Software - Infrastructure
2     IBM      Information Technology Services
3     WMT                      Discount Stores
4       V                      Credit Services
5      PG        Household & Personal Products
6     JNJ         Drug Manufacturers - General
7     UNH                     Healthcare Plans
8    INTC                       Semiconductors
9     XOM                 Oil & Gas Integrated
10     GS                      Capital Markets
11    JPM                  Banks - Diversified
12     HD              Home Improvement Retail
13    MCD                          Restaurants
14     BA                  Aerospace & Defense
15    CAT  Farm & Heavy Construction Machinery
16    CVX                 Oil & Gas Integrated
17   CSCO              Communication Equipment
18     KO            Beverages - Non-Alcoholic
19    DIS                        Entertainment
20    MMM    

Extract the ev/ebita ratio + industry

In [ ]:
import pandas as pd
import yfinance as yf
from yahoo_fin.stock_info import get_stats_valuation, tickers_dow

# Fetching the list of Dow 30 tickers
DOW_30_TICKER = tickers_dow()

# Initialize a DataFrame to hold the combined data
combined_data = []

# Fetch industry and EV/EBITDA for each ticker
for ticker in DOW_30_TICKER:
    # Fetch industry using yfinance
    t = yf.Ticker(ticker)
    info = t.info
    industry = info.get('industry', 'N/A')

    # Fetch EV/EBITDA ratio using yahoo_fin
    temp = get_stats_valuation(ticker)
    temp = temp.iloc[:,:2]  # Assuming the EV/EBITDA is in the first two columns
    temp.columns = ["Attribute", "Recent"]
    ev_ebitda = temp[temp["Attribute"] == "Enterprise Value/EBITDA"]["Recent"].values
    ev_ebitda_ratio = ev_ebitda[0] if len(ev_ebitda) > 0 else 'N/A'

    # Combine the data
    combined_data.append([ticker, industry, ev_ebitda_ratio])

# Convert the combined data into a DataFrame
combined_df = pd.DataFrame(combined_data, columns=['Ticker', 'Industry', 'EV/EBITDA'])

# Display the DataFrame
print(combined_df)


   Ticker                             Industry EV/EBITDA
0    AAPL                 Consumer Electronics     21.67
1    AMGN         Drug Manufacturers - General     19.49
2     AXP                      Credit Services       NaN
3      BA                  Aerospace & Defense     69.71
4     CAT  Farm & Heavy Construction Machinery     12.11
5     CRM               Software - Application     28.96
6    CSCO              Communication Equipment     10.08
7     CVX                 Oil & Gas Integrated      9.76
8     DIS                        Entertainment     18.41
9     DOW                            Chemicals     12.73
10     GS                      Capital Markets       NaN
11     HD              Home Improvement Retail     15.72
12    HON                        Conglomerates     15.41
13    IBM      Information Technology Services     14.58
14   INTC                       Semiconductors     18.40
15    JNJ         Drug Manufacturers - General     17.41
16    JPM                  Bank

For experimentation sake we are extracting S&P 500 tickers from Wikipedia, this would not be ideal because the ranking changes quarterly but for now we will do this

In [ ]:


# Fetch the list of S&P 500 companies' tickers from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
rows = table.findAll('tr')[1:]
tickers = [row.findAll('td')[0].text.strip() for row in rows]

# Display the list of tickers
print(tickers)


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DH

In [ ]:
import pandas as pd
import yfinance as yf
from yahoo_fin.stock_info import get_stats_valuation
import requests
from bs4 import BeautifulSoup

# Fetch the list of S&P 500 companies' tickers from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table', {'class': 'wikitable sortable'})
rows = table.findAll('tr')[1:]
SP_500_TICKERS = [row.findAll('td')[0].text.strip() for row in rows]

# Initialize a DataFrame to hold the combined data
combined_data = []

# Fetch industry and EV/EBITDA for each ticker
for ticker in SP_500_TICKERS:
    # Fetch industry using yfinance
    t = yf.Ticker(ticker)
    info = t.info
    industry = info.get('industry', 'N/A')

    # Fetch EV/EBITDA ratio using yahoo_fin
    temp = get_stats_valuation(ticker)
    temp = temp.iloc[:,:2]  # Assuming the EV/EBITDA is in the first two columns
    temp.columns = ["Attribute", "Recent"]
    ev_ebitda = temp[temp["Attribute"] == "Enterprise Value/EBITDA"]["Recent"].values
    ev_ebitda_ratio = ev_ebitda[0] if len(ev_ebitda) > 0 else 'N/A'

    # Combine the data
    combined_data.append([ticker, industry, ev_ebitda_ratio])

# Convert the combined data into a DataFrame
combined_df = pd.DataFrame(combined_data, columns=['Ticker', 'Industry', 'EV/EBITDA'])

# Display the DataFrame
print(combined_df)


Data not found for BF.B, skipping.
    Ticker                                Industry EV/EBITDA
0      MMM                           Conglomerates     -9.17
1      AOS          Specialty Industrial Machinery     14.25
2      ABT                         Medical Devices     22.86
3     ABBV            Drug Manufacturers - General     26.26
4      ACN         Information Technology Services     21.50
..     ...                                     ...       ...
497    YUM                             Restaurants     19.14
498   ZBRA                 Communication Equipment     15.93
499    ZBH                         Medical Devices     14.00
500   ZION                          Banks—Regional       NaN
501    ZTS  Drug Manufacturers—Specialty & Generic     24.28

[502 rows x 3 columns]


In [ ]:
#download the combineddf for visual analysis

#from google.colab import files

# Save the DataFrame to an Excel file
#combined_df.to_excel('combined_data.xlsx', index=False)

# Download the file to your local drive
#files.download('combined_data.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Ignore the following parts and code accordingly.

In [ ]:
# TRAIN_START_DATE = '2009-04-01'
# TRAIN_END_DATE = '2021-01-01'
# TEST_START_DATE = '2021-01-01'
# TEST_END_DATE = '2022-06-01'

# TRAIN_START_DATE = '2010-01-01'
# TRAIN_END_DATE = '2021-10-01'
# TEST_START_DATE = '2021-10-01'
# TEST_END_DATE = '2023-03-01'

# df = YahooDownloader(start_date = TRAIN_START_DATE,
#                      end_date = TEST_END_DATE,
#                      ticker_list = DOW_30_TICKER).fetch_data()

df = si.get_data(ticker, start_date = '2020-01-01', end_date = '2024-01-01', index_as_date = True)
                  # interval = “1mo”)

In [ ]:
df.head()

,open,high,low,close,adjclose,volume,ticker
2020-01-02,55.230000,55.540001,53.639999,53.750000,43.648018,5145200,DOW
2020-01-03,53.110001,53.599998,52.410000,52.419998,42.567974,5004300,DOW
2020-01-06,52.320000,52.880001,52.000000,52.209999,42.397438,4293100,DOW
2020-01-07,52.290001,53.064999,51.990002,52.189999,42.381203,5122800,DOW
2020-01-08,52.410000,53.169998,51.919998,52.730000,42.819714,3629700,DOW


In [ ]:
df.tail()

,open,high,low,close,adjclose,volume,ticker
2023-12-22,55.090000,55.660000,54.849998,55.119999,55.119999,2341900,DOW
2023-12-26,55.209999,55.895000,55.119999,55.709999,55.709999,2266200,DOW
2023-12-27,55.689999,55.858002,55.470001,55.709999,55.709999,2535300,DOW
2023-12-28,55.590000,55.700001,55.160000,55.230000,55.230000,2172600,DOW
2023-12-29,55.099998,55.259998,54.680000,54.840000,54.840000,2271400,DOW


In [ ]:
df.shape

(1006, 7)

In [ ]:
df.sort_values(['date','tic']).head()

KeyError: 'date'

In [ ]:
len(df.tic.unique())

In [ ]:
df.tic.value_counts()

# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [ ]:
#  INDICATORS = ['macd',
#                'rsi_30',
#                'cci_30',
#                'dx_30',]
INDICATORS = [
               'ebitda_30']

In [ ]:
# fe = FeatureEngineer(use_technical_indicator=True,
#                      tech_indicator_list = INDICATORS,
#                      use_turbulence=True,
#                      user_defined_feature = False)

fe = FeatureEngineer(use_financial_indicator=True,
                     fundamental_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

In [ ]:
processed.sample(5)

<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

In [ ]:
stock_dimension = len(processed.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


In [ ]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    "print_verbosity":5

}

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

* In this notebook, we are training and validating 3 agents (A2C, PPO, DDPG) using Rolling-window Ensemble Method ([reference code](https://github.com/AI4Finance-LLC/Deep-Reinforcement-Learning-for-Automated-Stock-Trading-Ensemble-Strategy-ICAIF-2020/blob/80415db8fa7b2179df6bd7e81ce4fe8dbf913806/model/models.py#L92))

In [ ]:
rebalance_window = 63 # rebalance_window is the number of days to retrain the model
validation_window = 63 # validation_window is the number of days to do validation and trading (e.g. if validation_window=63, then both validation and trading period will be 63 days)

ensemble_agent = DRLEnsembleAgent(df=processed,
                 train_period=(TRAIN_START_DATE,TRAIN_END_DATE),
                 val_test_period=(TEST_START_DATE,TEST_END_DATE),
                 rebalance_window=rebalance_window,
                 validation_window=validation_window,
                 **env_kwargs)



In [ ]:
A2C_model_kwargs = {
                    'n_steps': 5,
                    'ent_coef': 0.005,
                    'learning_rate': 0.0007
                    }

PPO_model_kwargs = {
                    "ent_coef":0.01,
                    "n_steps": 2048,
                    "learning_rate": 0.00025,
                    "batch_size": 128
                    }

DDPG_model_kwargs = {
                      #"action_noise":"ornstein_uhlenbeck",
                      "buffer_size": 10_000,
                      "learning_rate": 0.0005,
                      "batch_size": 64
                    }

timesteps_dict = {'a2c' : 10_000,
                 'ppo' : 10_000,
                 'ddpg' : 10_000
                 }

In [ ]:
df_summary = ensemble_agent.run_ensemble_strategy(A2C_model_kwargs,
                                                 PPO_model_kwargs,
                                                 DDPG_model_kwargs,
                                                 timesteps_dict)

In [ ]:
df_summary

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

In [ ]:
unique_trade_date = processed[(processed.date > TEST_START_DATE)&(processed.date <= TEST_END_DATE)].date.unique()

In [ ]:
df_trade_date = pd.DataFrame({'datadate':unique_trade_date})

df_account_value=pd.DataFrame()
for i in range(rebalance_window+validation_window, len(unique_trade_date)+1,rebalance_window):
    temp = pd.read_csv('results/account_value_trade_{}_{}.csv'.format('ensemble',i))
    df_account_value = pd.concat([df_account_value,temp],ignore_index=True)
sharpe=(252**0.5)*df_account_value.account_value.pct_change(1).mean()/df_account_value.account_value.pct_change(1).std()
print('Sharpe Ratio: ',sharpe)
df_account_value=df_account_value.join(df_trade_date[validation_window:].reset_index(drop=True))

In [ ]:
df_account_value.head()

In [ ]:
%matplotlib inline
df_account_value.account_value.plot()

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
df_dji_ = get_baseline(
        ticker="^DJI",
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(df_dji_, value_col_name = 'close')

In [ ]:
df_dji = pd.DataFrame()
df_dji['date'] = df_account_value['date']
df_dji['dji'] = df_dji_['close'] / df_dji_['close'][0] * env_kwargs["initial_amount"]
print("df_dji: ", df_dji)
df_dji.to_csv("df_dji.csv")
df_dji = df_dji.set_index(df_dji.columns[0])
print("df_dji: ", df_dji)
df_dji.to_csv("df_dji+.csv")

df_account_value.to_csv('df_account_value.csv')


<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:


# print("==============Compare to DJIA===========")
# %matplotlib inline
# # S&P 500: ^GSPC
# # Dow Jones Index: ^DJI
# # NASDAQ 100: ^NDX
# backtest_plot(df_account_value,
#               baseline_ticker = '^DJI',
#               baseline_start = df_account_value.loc[0,'date'],
#               baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])
df.to_csv("df.csv")
df_result_ensemble = pd.DataFrame({'date': df_account_value['date'], 'ensemble': df_account_value['account_value']})
df_result_ensemble = df_result_ensemble.set_index('date')

print("df_result_ensemble.columns: ", df_result_ensemble.columns)

# df_result_ensemble.drop(df_result_ensemble.columns[0], axis = 1)
print("df_trade_date: ", df_trade_date)
# df_result_ensemble['date'] = df_trade_date['datadate']
# df_result_ensemble['account_value'] = df_account_value['account_value']
df_result_ensemble.to_csv("df_result_ensemble.csv")
print("df_result_ensemble: ", df_result_ensemble)
print("==============Compare to DJIA===========")
result = pd.DataFrame()
# result = pd.merge(result, df_result_ensemble, left_index=True, right_index=True)
# result = pd.merge(result, df_dji, left_index=True, right_index=True)
result = pd.merge(df_result_ensemble, df_dji, left_index=True, right_index=True)
print("result: ", result)
result.to_csv("result.csv")
result.columns = ['ensemble', 'dji']

%matplotlib inline
plt.rcParams["figure.figsize"] = (15,5)
plt.figure();
result.plot();